In [1]:
! pip install transformers torch datasets rouge_score jiwer

/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'


In [22]:
import torch
from datasets import load_dataset, load_metric, list_datasets, list_metrics
from transformers import BertTokenizer, EncoderDecoderModel
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [23]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased') # initialize Bert2Bert from pre-trained checkpoints
# # forward
# input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
# outputs = model(input_ids=input_ids, decoder_input_ids=input_ids)
# # training
# outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, labels=input_ids)
# loss, logits = outputs.loss, outputs.logits
# # save and load from pretrained
# model.save_pretrained("bert2bert")
# model = EncoderDecoderModel.from_pretrained("bert2bert")
# # generation
# generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.pad_token_id)

In [24]:
csv_file = 'seq2seq_4335716.csv'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

val_data = load_dataset('csv', data_files=csv_file, split='train[90%:]')
train_data = load_dataset('csv', data_files=csv_file, split='train[:90%]')
print(val_data)
print(train_data)

Using custom data configuration default-711f9814101edcda
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-711f9814101edcda/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
Using custom data configuration default-711f9814101edcda
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-711f9814101edcda/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


Dataset({
    features: ['input_seq', 'output_seq'],
    num_rows: 50
})
Dataset({
    features: ['input_seq', 'output_seq'],
    num_rows: 451
})


In [25]:
val_data['output_seq']

['STOP_USING TSPATULA STOVE_MED CHEF_CHECK LBOIL STOVE_MED',
 'USE TSPATULA STOVE_MED PUT IN2e0UIJI STOVE_MED',
 'MOVE_CONTENTS STOVE_MED STOVE_MEDLOW USE TPAN STOVE_MEDLOW STOP_USING TPAN STOVE_MED STOP_USING TSPATULA STOVE_MED CHEF_CHECK LTIME STOVE_MEDLOW',
 'USE TSPATULA STOVE_MEDLOW PUT IzrIHcuDJ STOVE_MEDLOW PUT IrXTUISnn STOVE_MEDLOW',
 'MOVE_CONTENTS STOVE_MED SERVE MOVE_CONTENTS STOVE_MEDLOW SERVE STOP_USING TPAN STOVE_MEDLOW STOP_USING TSPATULA STOVE_MEDLOW PUT Is_lS1dmt SERVE',
 'USE TBOWL COUNTER1 PUT IzrIHcuDJ COUNTER1 PUT IzPYYTv5e COUNTER1 PUT IdCkp3LYx COUNTER1 PUT Is_lS1dmt COUNTER1',
 'MOVE_CONTENTS COUNTER1 OVEN_MED USE TBAKE_DISH OVEN_MED STOP_USING TBOWL COUNTER1 CHEF_CHECK LTIME OVEN_MED',
 'MOVE_CONTENTS OVEN_MED COUNTER1 USE TPAN COUNTER1 STOP_USING TBAKE_DISH OVEN_MED PUT IoskwJgPz COUNTER1 PUT IlGA3C7DK COUNTER1 PUT IraFb0aXd COUNTER1 PUT I6WhEfvid COUNTER1',
 'MOVE_CONTENTS COUNTER1 STOVE_MED MOVE_CONTENTS OVEN_MED STOVE_MED USE TPAN STOVE_MED STOP_USING TPAN

In [26]:
batch_size = 16  # 4 but change to 16 for full training
encoder_max_length = 128
decoder_max_length = 128
device = 'cuda' if torch.cuda.is_available() else 'cpu'


def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    tok_params = {
        'padding': 'max_length',
        'truncation': True,
        'max_length': encoder_max_length,
    }
    inputs = tokenizer(batch['input_seq'], **tok_params)
    outputs = tokenizer(batch['output_seq'], **tok_params)

    batch['input_ids'] = inputs.input_ids
    batch['attention_mask'] = inputs.attention_mask
    batch['decoder_input_ids'] = outputs.input_ids
    batch['decoder_attention_mask'] = outputs.attention_mask
    batch['labels'] = outputs.input_ids.copy()

    # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
    # We have to make sure that the PAD token is ignored
    batch['labels'] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch['labels']
    ]

    return batch

In [ ]:
# only use 32 training examples for notebook - COMMENT LINE FOR FULL TRAINING
# train_data = train_data.select(range(32))

train_data = train_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
#     remove_columns=['name', 'note'],
)
train_data.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
)

# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data = val_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
#     remove_columns=['name', 'note'],
)
val_data.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
)

ed_model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased')

# set special tokens
ed_model.config.decoder_start_token_id = tokenizer.bos_token_id
ed_model.config.eos_token_id = tokenizer.eos_token_id
ed_model.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
ed_model.config.vocab_size = ed_model.config.decoder.vocab_size
ed_model.config.max_length = 142
ed_model.config.min_length = 56
ed_model.config.no_repeat_ngram_size = 3
ed_model.config.early_stopping = True
ed_model.config.length_penalty = 2.0
ed_model.config.num_beams = 4


# load wer for validation
wer = load_metric('wer')


def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    wer_output = wer.compute(predictions=pred_str, references=label_str)

    return {
        'wer': round(wer_output, 4),
    }


# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir='./',
    evaluation_strategy='steps',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_steps=500,  # 2 or set to 1000 for full training
    save_steps=500,  # 16 or set to 500 for full training
    eval_steps=500,  # 4 or set to 8000 for full training
    warmup_steps=500,  # 1 or set to 2000 for full training
    max_steps=2500,  # 16 or comment for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=torch.cuda.is_available(),
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=ed_model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/csv/default-711f9814101edcda/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-9fd16f8276b06b93.arrow
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattent

Step,Training Loss,Validation Loss


# Backup

In [ ]:
ed_model.save_pretrained("recipe_bert_seq2seq")
!aws s3 cp --recursive recipe_bert_seq2seq s3://uatt-data/uri/
!rm -rf recipe_bert_seq2seq

# Tests

In [ ]:
input_text="Cook chicken until golden brown ."
input_ids = torch.tensor(tokenizer.encode(input_text, add_special_tokens=True)).unsqueeze(0).to(device)  # Batch size 1
generated = ed_model.generate(input_ids, decoder_start_token_id=ed_model.config.decoder.pad_token_id)

In [ ]:
output_ids = generated.to('cpu').numpy().reshape(-1)
tokenizer.convert_ids_to_tokens(output_ids)

In [ ]:
!aws s3 cp transformers-test.ipynb s3://uatt-notebooks/